In [1]:
import gzip
import json
from dateutil.parser import parse as pt
import pandas as pd
import numpy as np
import random
from tqdm import tqdm, trange

In [2]:
random.seed(5)

## 读取数据转为dataframe

In [3]:
%time
data_name = 'Toys_and_Games'

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


In [4]:
'''
自己编写的
'''
g = gzip.open('../data/reviews_'+data_name+'_5.json.gz', 'r')
reviews = []
for l in g:
    reviews.append(json.loads(l.decode()[:-1]))
values = dict()
ks = ['reviewerID', 'asin', 'reviewText', 'reviewTime', 'unixReviewTime']
for k in ks:
    values[k] = []
for i in range(len(reviews)):
    try:
        for k in ks:
#             if k == 'reviewTime':
#                 values[k].append(pt(reviews[i][k]))
#             else:
            values[k].append(reviews[i][k])
    except:
        print(reviews[i])
review_datas = pd.DataFrame(values)

In [5]:
# 物品数。user数
len(set(review_datas['asin'])),len(set(review_datas['reviewerID']))

(11924, 19412)

In [6]:
import nltk
import re

In [7]:
stopwords = nltk.corpus.stopwords.words('english') + list(';.~!@#$:%^&*(()_+=-[])/|\'\"?<>,`\\1234567890')

In [8]:
def help_f_cut_stop_word(x):
    x = x.lower()
    x = re.sub(r'([;\.~\!@\#\$\%\^\&\*\(\(\)_\+\=\-\[\]\)/\|\'\"\?<>,`\\])','',x)
    ss = ""
    words = x.split(' ')
    for w in words:
        if (w in stopwords):
            pass
        else:
            ss += ' ' + w
    return ss.lower().strip()

review_datas['reviewText'] = review_datas['reviewText'].map(help_f_cut_stop_word)

In [9]:
review_datas.head(2)

,reviewerID,asin,reviewText,reviewTime,unixReviewTime
0,A1VXOAVRGKGEAK,0439893577,like item pricing granddaughter wanted mark wa...,"01 29, 2014",1390953600
1,A8R62G708TSCM,0439893577,love magnet easel great moving different areas...,"03 28, 2014",1395964800


## 设置seq的最大长度为均值长度

In [10]:
r_lens = []
for i in review_datas['reviewText']:
    r_lens.append(len(i.split(' ')))
max_review_len = max(r_lens)
max_review_len

3084

In [11]:
mean_review_len = int(sum(r_lens)/len(r_lens))
max_review_len = mean_review_len
mean_review_len

51

In [12]:
def cutWord(x):
    if (len(x.split(' ')) > mean_review_len):
        x = x[:mean_review_len]
    return x
'''
记录最大的review length
'''
review_datas['reviewText'] = review_datas['reviewText'].map(cutWord)

## 按时间排序数据

In [13]:
review_datas = review_datas.sort_values(by='unixReviewTime', ascending=True)

## 添加时间戳

In [14]:
review_datas['timeBin'] = None

In [15]:
time_scope = [j for j in range(1997,2016)]
dataBins = [pt(str(i)+"-1-1").timestamp() for i in time_scope]
len(time_scope)
temporal_datas = [pd.DataFrame(columns=('reviewerID', 'asin', 'reviewText', 'reviewTime', 'unixReviewTime')) for _ in range(len(time_scope)-1)]

In [16]:
for t in range(0,len(dataBins)-1):
        temporal_datas[t] = review_datas[(review_datas['unixReviewTime'] >= dataBins[t]) & (review_datas['unixReviewTime'] < dataBins[t+1])]
        temporal_datas[t].loc[:, 'timeBin'] = t
        review_datas.loc[(review_datas['unixReviewTime'] >= dataBins[t]) & (review_datas['unixReviewTime'] < dataBins[t+1]), 'timeBin'] = t

/home/rjx/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
review_datas.sample(4)

,reviewerID,asin,reviewText,reviewTime,unixReviewTime,timeBin
103905,A12993071VTG5G,B0050OTNMC,bought gift year old loves dress loved every o...,"01 8, 2013",1357603200,16
42157,A2TFBF9OA0XVFW,B000RA76S2,great device low cost works exactly described ...,"12 21, 2012",1356048000,15
160153,A3P79VQHXG3QRL,B00DKCI1P6,grandson two bought go go smart wheels parents...,"05 5, 2014",1399248000,17
3036,A29VRZ6F7Z03XA,B00000IRTI,almost year old really enjoys since makes easy...,"01 8, 2006",1136678400,9


### 每个时间段的数据

In [18]:
for i in range(len(time_scope)-1):
    print(time_scope[i],' : ',review_datas[review_datas['timeBin'] == i].shape[0])

1997  :  0
1998  :  0
1999  :  0
2000  :  28
2001  :  128
2002  :  245
2003  :  493
2004  :  1031
2005  :  2035
2006  :  1172
2007  :  1484
2008  :  2820
2009  :  3977
2010  :  7381
2011  :  14946
2012  :  29815
2013  :  64601
2014  :  37441


## 处理元数据

In [19]:
meda_g = gzip.open('../data/meta_'+data_name+'.json.gz', 'r')

In [20]:
rr = []
for l in meda_g:
    l = l.decode()[:-1]
    try:
        rr.append(eval(l))
    except:
        print(l.decode()[:-1].replace('\'', '\"'))
values = dict()
ks = ['asin', 'categories']
for k in ks:
    values[k] = []
for i in range(len(rr)):
    try:
        for k in ks:
            values[k].append(rr[i][k])
    except:
        print(rr[i])

In [21]:
meta_datas = pd.DataFrame(values)
meta_datas.set_index('asin', inplace=True)

In [22]:
 len(meta_datas)

336072

In [23]:
meta_datas.sample(3)

,categories
asin,
B002UKQS7C,"[[Toys & Games, Electronics for Kids, Electron..."
B0046W1CBE,"[[Toys & Games, Dolls & Accessories, Doll Acce..."
B002B2YH8Q,"[[Toys & Games, Games, Trading Card Games, Sin..."


## 构造Query

In [24]:
# ts = [['CDs & Vinyl', 'Classical', 'Chamber Music'], ['Musical Instruments', 'Instrument Accessories', 'General Accessories']]
ts = [['Musical Instruments', 'fdsaf'], ['fasd', 'jghjh']]
def get_query(x):
    qs = list()
    for sub_cat_list in x:
        '''
        依照原来的论文需要修改
        '''
        if (len(sub_cat_list) <= 1):
            continue
        qs.append(sub_cat_list)
    '''
    remove duplicate
    '''
    finalQs = []
    
    for q in qs:
        
        Q_words = ' '.join(q).lower().replace(' & ', ' ').replace(',', '').strip().split(' ')
    #     words = set()
    #     for i in range(len(q)-1, -1, -1):
    #         if  (q[i] not in words):
    #             finalQ = q[i] + " " + finalQ
    #             words.add(q[i])
        finalQ = ''
        words = set()
        for i in range(len(Q_words)-1, -1, -1):
            if (Q_words[i] not in words):
                finalQ = Q_words[i] + ' ' + finalQ
                words.add(Q_words[i])
        finalQs.append( finalQ.strip())
    return finalQs


get_query(ts)

['musical instruments fdsaf', 'fasd jghjh']

In [25]:
meta_datas['query'] = meta_datas['categories'].map(get_query)

In [34]:
meta_datas

,categories,query
asin,,
0000191639,"[[Toys & Games, Puzzles, Jigsaw Puzzles]]",[toys games jigsaw puzzles]
0005069491,[[Toys & Games]],[]
0076561046,"[[Toys & Games, Learning & Education, Flash Ca...",[toys games learning education flash cards]
0131358936,"[[Toys & Games, Learning & Education, Mathemat...",[toys games learning education mathematics cou...
0133642984,"[[Toys & Games, Learning & Education, Mathemat...",[toys games learning education mathematics cou...
0279515766,"[[Toys & Games, Dolls & Accessories, Dolls]]",[toys games accessories dolls]
0375829695,"[[Toys & Games, Puzzles, Jigsaw Puzzles]]",[toys games jigsaw puzzles]
037585746X,"[[Toys & Games, Arts & Crafts, Drawing & Paint...",[toys games arts crafts painting supplies draw...
0425066169,"[[Toys & Games, Games, Board Games]]",[toys board games]


In [18]:
import pickle
data_name = 'Electronics'
with open('../input_data/meta_datas_time_bin_word'+data_name+'.bin', 'rb+') as f:
    meta_datas = pickle.load(f)

In [24]:
tt = meta_datas.sample(10)
tt.iloc[0][0], tt.iloc[0][1]

([['Electronics', 'Camera & Photo', 'Bags & Cases', 'Lens Cases']],
 ['electronics camera photo bags lens cases'])

In [ ]:
'''
记录最大的query length
'''
q_lens = []
for i in meta_datas['query']:
    for q in i:
        q_lens.append(len(q.split(' ')))
max_query_len = max(q_lens)
max_query_len

### 保存数据

In [ ]:
import pickle
'''
保存文件
'''
with open('../input_data/review_datas_time_bin_word'+data_name+'.bin', 'wb+') as f:
    pickle.dump(review_datas,f)
with open('../input_data/temporal_datas_time_bin_word'+data_name+'.bin', 'wb+') as f:
    pickle.dump(temporal_datas,f)
with open('../input_data/meta_datas_time_bin_word'+data_name+'.bin', 'wb+') as f:
    pickle.dump(meta_datas,f)

In [ ]:
review_datas.sample(3)

In [ ]:
for i in range(len(meta_datas)):
    if (len(meta_datas.iloc[i]['query']) > 4):
        print(meta_datas.iloc[i]['query'])
        break


In [ ]:
meta_datas.head(2)

### 构造train batch

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
class MyData(Dataset):
    def __init__(self, reviewData, metaData, neg_sample_num, max_query_len, max_review_len, time_num, weights = True):
        
        
        self.id2user = dict()
        self.user2id = dict()
        
        self.id2product = dict()
        self.product2id = dict()
        
        self.product2query = dict()
        
        # query
        self.word2id = dict()
        self.id2word = dict()
        
        
#         self.userText = dict()
        
        self.userReviews = dict()
        self.userReviewsCount = dict()
        self.userReviewsCounter = dict()
        self.userReviewsTest = dict()
        
        
        self.nes_weight = []
        self.word_weight = []
        self.max_review_len = max_review_len
        self.max_query_len = max_query_len
        self.neg_sample_num = neg_sample_num
        
        self.time_num = time_num
        self.time_data = []
        
        self.init_dict(reviewData, metaData)
        

        self.train_data = []
        self.test_data = []
        self.eval_data = []

        self.init_dataset(reviewData, weights)
        self.init_sample_table()
    def init_dict(self, reviewData, metaData):
        for i in range(self.time_num):
            self.time_data.append([])
        
        uid = 0
        us = set(reviewData['reviewerID'])
        pr = set()
        words = set()
        for u in us:
            # 只有两个购买记录 不够验证和测试
            asins = list(reviewData[reviewData['reviewerID'] == u]['asin'])
            if (len(asins) <= 2):
                continue

            self.id2user[uid] = u
            self.user2id[u] = uid

            # 得到每个用户购买物品记录
            pr.update(asins)
            self.userReviews[uid] = asins
            #　最后一个购买的物品做测试集
            self.userReviewsTest[uid] = asins[-1]
            words.update(set(' '.join(list(review_datas[review_datas['reviewerID'] == u]['reviewText'])).split()))
#             reviewTexts += list(reviewData[reviewData['reviewerID'] == u]['reviewText'])
            uid += 1
            if uid % 100 == 0:
                with open (r'out.txt','a+') as ff:
                    ff.write(str(len(us))+' uid: '+str(uid)+'\n')
        self.userNum = uid
        
        pid = 0
#         words = set()
        for p in pr:
            if pid % 300 == 0:
                with open (r'out.txt','a+') as ff:
                    ff.write(str(len(pr)) + ' pid:'+str(pid)+'\n')
            try:
                '''
                判断这个product是否有query
                '''
                if (len(metaData.loc[p]['query']) > 0):
                    self.product2query[p] = metaData.loc[p]['query']
                    words.update(' '.join(metaData.loc[p]['query']).split(' '))
            except:
                pass
            self.id2product[pid] = p
            self.product2id[p] = pid
            pid += 1
            
        self.productNum = pid
        self.nes_weight = np.zeros(self.productNum)
        self.queryNum = len(self.product2query)
        
        wi = 0
        self.word2id['<pad>'] = wi
        self.id2word[wi] = '<pad>'
        wi += 1
        for w in words:
            if(w==''):
                continue
            self.word2id[w] = wi
            self.id2word[wi] = w
            wi += 1
        self.wordNum = wi
        self.word_weight = np.zeros(wi)
    def init_dataset(self, reviewData,weights=True):
        try:
            self.data_X = []
            for r in range(len(reviewData)):
                if r % 100 == 0:
                    with open (r'out.txt','a+') as ff:
                        ff.write(str(len(reviewData))+ ' review: '+str(r) + '\n')
                rc = reviewData.iloc[r]
                try:
                    uid = self.user2id[rc['reviewerID']]
                    pid_pos = self.product2id[rc['asin']]
                    time_bin_pos = int(rc['timeBin'])
                except:
                    # 这个user没有加入到字典，购买次数不到3次
                    continue

                text = rc['reviewText']

                try:
                    # 得到product的query数组
                    q_text_array_pos = self.product2query[self.id2product[pid_pos]]
                except:
                    '''
                    没有对应的query
                    '''
                    continue
                try:
                    text_ids, len_r= self.trans_to_ids(text, self.max_review_len)
                    # 设置product的负采样频率
                    self.nes_weight[pid_pos] += 1
                except:
                    continue
                # 遍历每个物品的每个query 得到一个(u, p, q, r)元组
                for qi in range(len(q_text_array_pos)):
                    try:
                        qids_pos, len_pos = self.trans_to_ids(q_text_array_pos[qi], self.max_query_len)
                    except:
                        break
                    self.data_X.append((uid, pid_pos, qids_pos, len_pos, text_ids, len_r, time_bin_pos))
                    try:
                        self.userReviewsCount[uid] += 1
                        self.userReviewsCounter[uid] += 1
                    except:
                        self.userReviewsCount[uid] = 1
                        self.userReviewsCounter[uid] = 1


            '''
            数据集合划分 ---> 取每个用户购买过的item的最后一个
            '''
            for r in self.data_X:
                # 只考虑有3个以上（uqi）三元组的user
                if self.userReviewsCount[r[0]] > 2:
                    t = self.userReviewsCounter[r[0]]
                    if (t == 0):
                        continue
                    elif (t == 2): # 倒数第二个
                        self.eval_data.append(r)
                    elif (t == 1): # 倒数第一个
                        self.test_data.append(r)
                    else:
                        self.train_data.append(r)
                        self.time_data[r[6]].append(r)
                    self.userReviewsCounter[r[0]] -= 1

            if weights is not False:
                wf = np.power(self.nes_weight, 0.75)
                wf = wf / wf.sum()
                self.weights = wf
                wf = np.power(self.word_weight, 0.75)
                wf = wf / wf.sum()
                self.word_weight = wf
        except e:
            with open (r'out.txt','a+') as ff:
                ff.write(str(e)+ '\n')
    def trans_to_ids(self, query, max_len, weight_cal = True):
        query = query.split(' ')
        qids = []
        for w in query:
            if w == '':
                continue
            qids.append(self.word2id[w])
            # 需要统计词频
            if weight_cal:
                self.word_weight[self.word2id[w]-1] += 1
        for _ in range(len(qids), max_len):
            qids.append(self.word2id['<pad>'])
        return qids, len(query)
    
    def neg_sample(self):
        neg_item = []
        neg_word = []
        for ii in range(self.neg_sample_num):
            neg_item.append(self.sample_table_item[np.random.randint(self.table_len_item)])
            neg_word.append(self.sample_table_word[np.random.randint(self.table_len_word)])
        return neg_item,neg_word
    
    def init_sample_table(self):
        table_size = 1e6
        count = np.round(self.weights*table_size)
        self.sample_table_item = []
        for idx, x in enumerate(count):
            self.sample_table_item += [idx]*int(x)
        self.table_len_item = len(self.sample_table_item)
        
        count = np.round(self.word_weight*table_size)
        self.sample_table_word = []
        for idx, x in enumerate(count):
            self.sample_table_word += [idx]*int(x)
        self.table_len_word = len(self.sample_table_word)
    
    def __getitem__(self, i):
        pos = self.train_data[i]
        neg = self.neg_sample()
        
        return pos, neg
    def get_time_data(self, time_bin, i):
        pos = self.time_data[time_bin][i]
        neg = self.neg_sample()
        return pos, neg
    def getTestItem(self, i):
        return self.test_data[i]
    def __len__(self):
        return len(self.train_data)

In [ ]:
data_loader = MyData(review_datas, meta_datas, 5, max_query_len, max_review_len, len(temporal_datas), weights = True)

In [ ]:
dl = DataLoader(data_loader, batch_size=5, shuffle=False, drop_last=False)

In [ ]:
ii = iter(dl)

In [ ]:
a = ii.next()

In [ ]:
len(a)

In [ ]:
len(a[0])

In [ ]:
a[0][0]

In [ ]:
# (uid, pid_pos, qids_pos, len_pos, text_ids, len_r, time_bin_pos)

In [ ]:
a[0][6]

In [ ]:
import pickle

In [ ]:
with open('../input_data/dataset_time_'+data_name+'.bin', 'wb+') as f:
    pickle.dump(data_loader,f)

In [49]:
print(len(data_loader.test_data))

192403


In [50]:
print(len(data_loader.train_data))

1421049


In [51]:
print(len(data_loader.data_X))

1805855


In [48]:
1429+7393

8822